# Phase 1: MNIST Handwritten Digit Dataset

In [40]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

In [ ]:
## basic transformations which are mandotory --> i.e converting to tensor and normalization

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# creating dataset objects - later we will also have dataloader objects

train_dataset = torchvision.datasets.MNIST(
    root = '/data',
    train = True,
    download = True,
    transform = transform
)

test_dataset = torchvision.datasets.MNIST(
    root = '/data',
    train = False,
    download = True,
    transform = transform
)

print (len(train_dataset))
print (len(test_dataset))

60000
10000


In [ ]:
# the dataset now contains images which can be accessed using train_dataset[x]
# each image also has a true label whih is the actual class of that label
# currently we are accessing the 69th image --> which turns out to be a zero

sample_image, sample_label = train_dataset[69]

print (sample_image.shape)
print (sample_label)

torch.Size([1, 28, 28])
0


In [ ]:
# lets also do it for test dataset
# funny enough we have zero as the 69th image in both the datasets

sample_image, sample_label = test_dataset[69]

print (sample_image.shape)
print (sample_label)

torch.Size([1, 28, 28])
0


# Phase 2: How to Define and Train the Discriminator Model


### Phase 2.1 Discriminator architecture

In [47]:
class Discriminator(nn.Module):

  def __init__ (self):
    super(Discriminator, self).__init__()

    self.first_conv_block = self._make_first_conv_block()
    self.second_conv_block = self._make_second_conv_block()
    self.final_classification_block = self._make_final_classification_block()


  def _make_first_conv_block(self):

    layers = []

    layers.append(
        nn.Conv2d(
            in_channels = 1,
            out_channels = 64,
            kernel_size = 3,
            stride = 2,
            padding = 1
        )
    )

    layers.append(
        nn.LeakyReLU(
            negative_slope = 0.2,
            inplace = True
        )
    )

    layers.append(
        nn.Dropout(
            p = 0.4
        )
    )

    return nn.Sequential(*layers)





  def _make_second_conv_block(self):

    layers = []

    layers.append(
        nn.Conv2d(
            in_channels = 64,
            out_channels = 64,
            kernel_size = 3,
            stride = 2,
            padding = 1
        )
    )

    layers.append(
        nn.LeakyReLU(
            negative_slope = 0.2,
            inplace = True
        )
    )

    layers.append(
        nn.Dropout(
            p = 0.4
        )
    )

    return nn.Sequential(*layers)






  def _make_final_classification_block (self):

    layers = []

    layers.append(nn.Flatten())

    layers.append(
        nn.Linear(
            in_features = 7*7*64,
            out_features = 1
        )
    )

    layers.append(nn.Sigmoid())

    return nn.Sequential(*layers)



  def forward (self, x):

    x = self.first_conv_block(x)
    x = self.second_conv_block(x)
    x = self.final_classification_block(x)

    return x


In [48]:
def create_discriminator():

  discriminator = Discriminator()


  # in article they have decided to use adam sgd and bce loss


  optimizer = optim.Adam(
      discriminator.parameters(),
      lr = 0.0002,
      betas = (0.5, 0.999)
  )

  criterion = nn.BCELoss()

  return discriminator, optimizer, criterion



discriminator, optimizer, criterion = create_discriminator()

In [51]:
total_params = sum(p.numel() for p in discriminator.parameters())
trainable_params = sum(p.numel() for p in discriminator.parameters() if p.requires_grad)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")


Total Parameters: 40,705
Trainable Parameters: 40,705


### Phase 2.2 Setup Training for Discriminator